In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager
!pip install pandas
!pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,664 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.n

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import numpy as np
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys

ModuleNotFoundError: No module named 'selenium'

In [ ]:
def print_current_date():
    now = datetime.now()
    current_date = now.strftime("%Y-%m-%d")
    return(current_date)


In [ ]:
def print_current_time():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")


    return(current_time)

In [ ]:
def device_time():
    import datetime
    import pytz
    utc_now = datetime.datetime.now(datetime.timezone.utc)
    ist_timezone = pytz.timezone('Asia/Kolkata')
    ist_now = utc_now.astimezone(ist_timezone)
    return ist_now.strftime("%H:%M:%S")

In [ ]:
def setup_driver():
    service = Service(executable_path=r'chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver= webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 15)  # Initialize WebDriverWait with a 10-second timeout
    return driver, wait

In [ ]:
def setup_driver_producturl(pr_name):
    service = Service(executable_path=r'chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    #chrome_options.add_argument('--disable-dev-shm-usage')
    driver= webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 15)  # Initialize WebDriverWait with a 10-second timeout
    driver.get('https://www.google.com/')
    search_box=wait.until(EC.presence_of_element_located((By.XPATH,'//*[@id="APjFqb"]')))
    key=pr_name.replace(" ","+")
    #print(key)
    search_box.send_keys(key)
    search_box.send_keys(Keys.RETURN)
    time.sleep(2)
    first_result = wait.until(EC.element_to_be_clickable((By.XPATH, "(//h3[@class='LC20lb MBeuO DKV0Md'])[1]")))
    first_result.click()
    time.sleep(1)
    return driver, wait

## Code for getting Product Container from main search card

In [ ]:
def avail_product(driver,url):
    #driver.get("https://www.samsung.com/in/washers-and-dryers/washing-machines/?front-loading+top-loading+semi-automatic")
    #print(url)
    #driver=click()
    driver.get(url)
    a=driver.find_element(By.XPATH ,'//*[contains(concat( " ", @class, " " ), concat( " ", "categorized-group", " " ))]')
    product_card=list(str(a.text).split("/n"))
    #print("Printing Shi:",shi)
    return product_card

##Function for Name of products from availble page


In [ ]:
def product_name_list(productcard):
    name=list(productcard[0].split("\n"))
    product_name=[]
    #print(len(name))
    for i in name:
      if len(i)>0 and (i[0].isdigit() or i[0]=="W"):
        if len(i)>30:
            product_name.append(i)
    #print("Printing Product name :",product_name)
    #print("______________________________________________")
    return product_name

## exctracting names from Product_name_list() and avail_product() function

In [ ]:
def total_avail_product_name(driver):
    name_list=[]
    for i in range(1,9):
        url="https://www.samsung.com/in/washers-and-dryers/washing-machines/?front-loading+top-loading+semi-automatic+18p0{}".format(i)
        Product_card=avail_product(driver,url)
        name=product_name_list(Product_card)
        #print("Printing Name:",name)
        #print(name_list)
        name_list.extend(name)
    return name_list

##Main Function which extract products name from website url which have a range of Price , returning name of total washing machines lister (available to Buy ) on Samsung Website

In [ ]:
def Filter(feature,bools):
    list_feature=[]
    for i in range(len(feature)):
      #print(feature[i].text)
      if (bools[i].text).lower()=='yes':
        list_feature.append(feature[i].text)
    return list_feature

##Feature of product

In [ ]:
def product_feature(driver):
     #driver.get('https://www.samsung.com/in/washers-and-dryers/washing-machines/ww7400t-11kg-black-ww11cgp44dsbtl/')
     soup=BeautifulSoup(driver.page_source,'html.parser')

     a=soup.findAll('div',{'class':['spec-highlight__detail','spec-highlight__detail-specs']})
     featur_list=[]
     for item in a:
       ab=item.findAll('strong')
       span=item.findAll('span')
       if len(ab)==len(span):
         featur_list.extend(Filter(ab,span))
         #print("=============================================================================")
     string=" ".join(featur_list)
     return string



##Getting Product Price

In [ ]:
def product_price(driver):
    try:
        price=driver.find_element(By.XPATH,'//*[@id="anchorContainer"]/div[2]/div[2]/div[1]/span').text
        return price[1:]

    except:
        return "NA"

Product Striked Price

In [ ]:
def product_strike_price(driver):
    try:
        striked_price=driver.find_element(By.XPATH,'//*[@id="anchorContainer"]/div[2]/div[2]/div[2]/div[2]/del').text
        return striked_price[1:]

    except:
        return "NA"

Dimensions of the Product

Weight of machine

In [ ]:
def machine_weight(driver):
    list_ul=0
    for i in range(1,7):
      try:
        name=driver.find_element(By.XPATH,f'//*[@id="specs"]/div[1]/ul/li[{i}]/div/strong').text
        #print(name)
        if name=="Net Weight":
            #print(name)
            list_ul=i
            break
      except Exception as e:
        print(e)
        pass
    if list_ul==0:
        return "NA"
    #print(list_ul)
    try:
        machi_weight=driver.find_element(By.XPATH,f'//*[@id="specs"]/div[1]/ul/li[{list_ul}]/div/span').text
        return machi_weight[:-2]
    except:
        return "NA"
#Done

Washing Capactiy


In [ ]:
def washing_capacity(driver):
    list_ul=0
    for i in range(1,7):
        try:
          name=driver.find_element(By.XPATH,f'//*[@id="specs"]/div[1]/ul/li[{i}]/div/strong').text
          if name=="Washing Capacity" or name=="Washing Capacity (kg)":
            list_ul=i
            break
        except Exception as e:
            print(e)

    if list_ul==0:
        return "NA"

    try:
        washing_cap=driver.find_element(By.XPATH,f'//*[@id="specs"]/div[1]/ul/li[{list_ul}]/div/span').text
        return washing_cap[:-3]
    except:
        return "NA"

In [ ]:
def dimensions(driver):
        list_ul=0
        for i in range(1,7):
            try:
                name=driver.find_element(By.XPATH,f'//*[@id="specs"]/div[1]/ul/li[{i}]/div/strong').text
                if name=="Physical specification" or name=="Net Dimension (WxHxD)":
                  list_ul=i
                  break
            except Exception as e:
                print(e)
                pass
        if list_ul==0:
            return "NA"

        dimension=driver.find_element(By.XPATH,f'//*[@id="specs"]/div[1]/ul/li[{list_ul}]/div/span').text

        dimension=dimension[:-2].split("x")
        #print(dimension)
        #print(len(dimension)) #//*[@id="specs"]/div[1]/ul/li[3]/div/span
        lis=[]
        if len(dimension):
           for i in dimension:
               #print(i)
               i.split()
               #i.replace(" ","")
               #print(i)
               lis.append(i[:])
        return lis

## Main fucntion

In [ ]:
def main():
    driver,wait=setup_driver()
    names=total_avail_product_name(driver) #getting name of all the product available on samsung webstie

    Product_name= []
    price_of_product = []
    Striked_price = []
    Width = []
    Height = []
    Depth = []
    Machin_weight= []
    washing_cap=[]
    feature=[]

    #names=names[:3]
    print(len(names))
    print("Running start at : ",device_time())
    count=1
    for name in names:
      try:
        driver,wait=setup_driver_producturl(name)

        Product_name.append(name)
        "-----------------------------------------------"
        featrues_of_product=product_feature(driver) #we have to append it
        feature.append(featrues_of_product)
        "-----------------------------------------------"
        price_of_prod=product_price(driver)
        price_of_product.append(price_of_prod)
        "-----------------------------------------------"
        waashing_capacity=washing_capacity(driver)
        washing_cap.append(waashing_capacity)
        "-----------------------------------------------"
        Striked_pric=product_strike_price(driver)
        Striked_price.append(Striked_pric)
        "-----------------------------------------------"
        dimention=dimensions(driver)
        #print(dimention)
        if dimention!="NA":
            try:
              width=dimention[0].strip()
              Width.append(width)
            except Exception as e:
              print("Exception at width:",e)
              Width.append("NA")
            "-----------------------------------------------"
            try:
              height=dimention[1].strip()
              Height.append(height)
            except Exception as e:
              print("Exception at hright:",e)
            "-----------------------------------------------"
            try :
              depth=dimention[2].strip()
              Depth.append(depth)
            except Exception as e:
              print("Exception at depth:",e)
              Depth.append("NA")
        else:
            Width.append("NA")
            Height.append("NA")
            Depth.append("NA")
        "-----------------------------------------------"
        weight_mach=machine_weight(driver)
        try:
          Machin_weight.append(weight_mach)
        except Exception as e:
          print("MAchine weight:",e)
          Machin_weight.append("NA")
        "-----------------------------------------------"
        print(f"Done {count} out of {len(names)}")
        count+=1

        '''print("----------------------------------------------------------------")
        print("Name of product",name)
        print("price of product",price_of_prod)
        print("Striked Price of Product: ",Striked_pric)
        print("Width of Product:",width)
        print("Height of Product:",height)
        print("Depth of Product:",depth)
        print("washing capacity is:",waashing_capacity)
        print("Weight of Machine :",weight_mach)
        print("Feature of Product",featrues_of_product)
        print("_________________________________________________________________")'''


      except Exception as e:
        print("error occure :- ",e)
        pass

    driver.quit()

    date=print_current_date()
    devicetime=device_time()

    print(len(Product_name),len(price_of_product),len(Striked_price),len(Width),len(Height),len(Depth),len(Machin_weight),len(washing_cap),len(feature))




    Scraped_data = {
        'Source': "Samsung", # Make 'Source' the same length
        'Model Name': Product_name,
        'Date': date ,  # Make 'Date' the same length
        'Time': devicetime ,  # Make 'Time' the same length
        'Original_MRP': Striked_price,
        'Discounted_Price': price_of_product,
        'Product Height': Height,  # Use the Height list directly
        'Product Width': Width,   # Use the Width list directly
        'Product Depth': Depth,   # Use the Depth list directly
        'Machine Weight': Machin_weight,
        'Washing Capacity': washing_cap,
        'Product Feature': feature
    }
    df=pd.DataFrame(Scraped_data)
    output_file = f'/content/Output_{date}_samsung.xlsx'

    try:
        existing_df = pd.read_excel(output_file)
        updated_df = pd.concat([existing_df, df], ignore_index=True)
    except FileNotFoundError:
        updated_df = df
    updated_df.to_excel(output_file, index=False)

    print(f"Data has been written to {output_file}")


## Main function

In [ ]:
main()

37
Running start at :  16:34:13
Done 1 out of 37
Done 2 out of 37
Done 3 out of 37
Done 4 out of 37
Done 5 out of 37
Done 6 out of 37
Done 7 out of 37
Done 8 out of 37
Done 9 out of 37
Done 10 out of 37
Done 11 out of 37
Done 12 out of 37
Done 13 out of 37
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="specs"]/div[1]/ul/li[1]/div/strong"}
  (Session info: chrome-headless-shell=126.0.6478.126); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55ff75990c7a <unknown>
#1 0x55ff75673e2c <unknown>
#2 0x55ff756c0661 <unknown>
#3 0x55ff756c0751 <unknown>
#4 0x55ff75704f64 <unknown>
#5 0x55ff756e35ed <unknown>
#6 0x55ff75702303 <unknown>
#7 0x55ff756e3363 <unknown>
#8 0x55ff756b3247 <unknown>
#9 0x55ff756b3b9e <unknown>
#10 0x55ff7595724b <unknown>
#11 0x55ff7595b2f1 <unknown>
#12 0x55ff75942afe <unknown>
#13 0x55ff7595be52 <unknown>
#14 0x

In [ ]:
#ke='6.0Kg+Semi+Automatic+with+Air+Turbo+Drying+System,+WT60R2000LL'
#driver,wait=setup_driver_producturl(ke)
#dimention=dimensions(driver)
#print(dimention)
#for i in dimention:
#    print(i.strip())

In [ ]:
driver,wait=setup_driver()

driver.get('https://www.samsung.com/in/washers-and-dryers/washing-machines/wt2500rm-twin-5-star-energy-rating-air-turbo-drying-system-rat-protection-6-5kg-blue-wt65r2200ll-tl/')

a=driver.find_element(By.OU,'//*[@id="tabPanelId0"]/div[1]/div[1]/div[1]')
print(driver.page_source)
print(a.page_source)
print(a.text)

driver.quit()

AttributeError: type object 'By' has no attribute 'OU'

In [ ]:
driver.get('https://www.samsung.com/in/washers-and-dryers/washing-machines/wt2500rm-twin-5-star-energy-rating-air-turbo-drying-system-rat-protection-6-5kg-blue-wt65r2200ll-tl/')

a=driver.find_element(By.XPATH,'//*[@id="tabPanelId0"]/h3')
print(a.text)

driver.quit()

In [ ]:
def avail_product(driver):
    #driver.get("https://www.samsung.com/in/washers-and-dryers/washing-machines/?front-loading+top-loading+semi-automatic")
    #print(url)
    driver.get('https://www.samsung.com/in/washers-and-dryers/washing-machines/?front-loading+top-loading+semi-automatic')
    #driver=click()

    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH,'//*[@id="content"]/div/div/div[3]/div/div[2]/section/div/div[2]/div[2]/div[2]/a'))
)
    try:
        element.click()
    except WebDriverException:
        print("Element is not clickable")


In [ ]:
driver,wait=setup_driver()
names=total_avail_product_name(driver) #getting name of all the product available on samsung websti
for name in names:
    print(name)

In [ ]:
name="8.0 kg AI Ecobubble™ Front Load Washing Machine with SmartThings & Wi-Fi, WW80T504DAX1"

setup_driver_producturl(name)